# Трендовые признаки (Нормализованные для Global ML Model)

**РЕФАКТОРИНГ ЗАВЕРШЕН** - Этот ноутбук теперь использует модуль `features.trend_features`

## Критические изменения:
- ❌ **УДАЛЕНЫ** абсолютные значения MA (sma_20, sma_50, ema_20...)
- ✅ **ДОБАВЛЕНЫ** относительные расстояния: `dist_to_sma_*`, `dist_to_ema_*`

## Формула нормализации:
```
dist_to_sma_50 = (close / SMA_50) - 1
```

## Нормализованные признаки:
- `dist_to_sma_20/50/200` - расстояние до SMA в процентах
- `dist_to_ema_20/50` - расстояние до EMA в процентах  
- `sma_20_slope_norm`, `sma_50_slope_norm` - наклон MA (нормализованный)
- `momentum_10/20` - log return за период
- `rsi_14` - RSI (уже нормализован 0-100)
- `trend_signal`, `trend_strength` - сигнал тренда

**Основной pipeline**: см. `06_feature_aggregator.ipynb`


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к корневой директории ML для импорта модулей
sys.path.insert(0, str(Path.cwd().parent))

# Импорт модуля трендовых признаков
from features.trend_features import (
    build_trend_features,
    dist_to_ma,
    rsi,
    momentum_normalized,
    trend_signal,
    trend_strength,
    TREND_FEATURE_COLUMNS
)

# Конфигурация путей
ML_ROOT = Path.cwd().parent
DATA_DIR = ML_ROOT / "data" / "processed"

print("✅ Библиотеки и модули загружены")
print(f"📂 Источник данных: {DATA_DIR}")


## Функции расчета трендовых признаков


In [ ]:
# Функции теперь импортированы из модуля features.trend_features
# 
# Доступные функции:
# - dist_to_ma(prices, ma_values) -> относительное расстояние до MA
# - rsi(prices, window=14) -> RSI (0-100)
# - momentum_normalized(prices, window) -> log return за период
# - trend_signal(ma_short, ma_long, threshold) -> сигнал тренда (-1, 0, 1)
# - trend_strength(ma_short, ma_long) -> сила тренда (нормализованная)
# - build_trend_features(df) -> все признаки сразу
#
# ВАЖНО: Все функции возвращают НОРМАЛИЗОВАННЫЕ значения!
# Абсолютные MA НЕ экспортируются в итоговый датасет.

print("✅ Функции трендовых признаков импортированы из модуля")
print(f"📋 Список признаков: {TREND_FEATURE_COLUMNS}")


## Загрузка и расчет признаков


In [ ]:
# Загрузка данных
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_ohlcv_returns.parquet")
print(f"📥 Загружено: {len(df)} записей для {ticker}")

# Расчет ВСЕХ нормализованных трендовых признаков одним вызовом
trend_features = build_trend_features(df)

print(f"✅ Трендовые признаки рассчитаны: {len(trend_features.columns)} столбцов")
print(f"\n📋 Признаки:")
for col in trend_features.columns:
    print(f"   • {col}")

print(f"\n📊 Пример данных (НОРМАЛИЗОВАННЫЕ):")
display(trend_features.tail(10))

# Сравнение: старый подход vs новый
print(f"\n⚠️ ВАЖНО: Обратите внимание на отличия:")
print(f"   РАНЬШЕ: sma_50 = 285.12 (абсолютная цена)")
print(f"   СЕЙЧАС: dist_to_sma_50 = 0.023 (относительное отклонение = +2.3%)")


## Сохранение результатов


In [ ]:
# Сохранение результатов
# ПРИМЕЧАНИЕ: Для полного pipeline используйте 06_feature_aggregator.ipynb
# Этот ноутбук сохраняет только трендовые признаки для демонстрации

OUTPUT_DIR = ML_ROOT / "data" / "features"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Сохраняем ТОЛЬКО нормализованные признаки (без абсолютных MA!)
output_path = OUTPUT_DIR / f"{ticker}_trend_features_normalized.parquet"
trend_features.to_parquet(output_path, index=False)

print(f"✅ Сохранено: {output_path}")
print(f"   Строк: {len(trend_features)}")
print(f"   Столбцов: {len(trend_features.columns)}")

# Проверка: убеждаемся что нет абсолютных MA
forbidden = ['sma_20', 'sma_50', 'sma_200', 'ema_20', 'ema_50']
found = [col for col in trend_features.columns if col in forbidden]
if found:
    print(f"\n❌ ОШИБКА: Найдены запрещенные столбцы: {found}")
else:
    print(f"\n✅ Проверка пройдена: нет абсолютных MA значений")
